# Importations

In [1]:
%matplotlib inline
from __future__ import division #Importer ça avant toute chose, autrement ne marche pas
import graphlab as gl
#import pandas as pd
gl.canvas.set_target('ipynb') # Pour afficher les graphes dans mon notebook plutôt que new tab
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from pylab import pcolor, show, colorbar, xticks, yticks
import seaborn as sns
from sklearn import preprocessing

# Fonctions

In [2]:
# Calculate the mean age for a given class
def round_age(df, i_class):
    return round(df["Age"][df["Pclass"]==i_class].mean())

In [3]:
# Get features list

# Data import

In [12]:
#Let's work with pandas then import in graphlab for use with models
df_data = pd.read_csv("./train.csv") #My training data

In [13]:
df_data_validation = pd.read_csv("./test.csv") #Data used for submission to Kaggle

# Constantes des modèles

In [ ]:
sex_coef = 2

# Data cleaning

If age is undefined and fare is zero then we set age to 0 (we suppose babies don't pay)

In [14]:
df_data.loc[(df_data.Age.isnull()) & (df_data["Fare"] == 0), "Age"] = 0
df_data_validation.loc[(df_data_validation.Age.isnull()) & (df_data_validation["Fare"] == 0), "Age"] = 0

In [15]:
df_data_validation.loc[(df_data_validation.Fare.isnull()), "Fare"] = 9

Calculate mean age (for a given class) for both data sets

In [16]:
mean_age = [round_age(df_data,i_class) for i_class in range(1,4,1)] #38, 29, 25
mean_age_valid = [round_age(df_data_validation,i_class) for i_class in range(1,4,1)] #41, 29, 24

Set undefined values to these mean age values

In [17]:
for i in range(1,4,1):
    df_data.loc[(df_data.Age.isnull()) & (df_data["Pclass"] == i), "Age"] = mean_age[i-1]
    df_data_validation.loc[(df_data_validation.Age.isnull()) & (df_data_validation["Pclass"] == i), "Age"] = mean_age_valid[i-1]


Normalization is done with sklearn preprocessing

In [18]:
x = df_data.values #Get only values from the dataframe
min_max_scaler = preprocessing.MinMaxScaler() #Create the scaling function
#x_scaled = min_max_scaler.fit_transform(x)
df_data["Age_n"] = min_max_scaler.fit_transform(df_data["Age"].reshape(-1,1))
df_data["Pclass_n"] = min_max_scaler.fit_transform(df_data["Pclass"].reshape(-1,1))
df_data["SibSp_n"] = min_max_scaler.fit_transform(df_data["SibSp"].reshape(-1,1))
df_data["Fare_n"] = min_max_scaler.fit_transform(df_data["Fare"].reshape(-1,1))

C:\Users\Michael\Anaconda2\envs\gl-env\lib\site-packages\sklearn\utils\validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\Michael\Anaconda2\envs\gl-env\lib\site-packages\sklearn\utils\validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [24]:
df_data.loc[df_data.Sex == "male", "Sex_n"] = 0
df_data.loc[df_data.Sex == "female", "Sex_n"] = 1

In [19]:
print df_data_validation["Fare"].isnull().sum()

0


In [20]:
del df_data["Name"]
del df_data["Ticket"]
del df_data["Cabin"]
del df_data["Embarked"]

In [21]:
y = df_data_validation.values #Get only values from the dataframe
min_max_scaler = preprocessing.MinMaxScaler() #Create the scaling function

In [23]:
df_data_validation["Age_n"] = min_max_scaler.fit_transform(df_data_validation["Age"].reshape(-1,1))
df_data_validation["Pclass_n"] = min_max_scaler.fit_transform(df_data_validation["Pclass"].reshape(-1,1))
df_data_validation["SibSp_n"] = min_max_scaler.fit_transform(df_data_validation["SibSp"].reshape(-1,1))
df_data_validation["Fare_n"] = min_max_scaler.fit_transform(df_data_validation["Fare"].reshape(-1,1))
df_data_validation.loc[df_data_validation.Sex == "male", "Sex_n"] = 0
df_data_validation.loc[df_data_validation.Sex == "female", "Sex_n"] = 1

C:\Users\Michael\Anaconda2\envs\gl-env\lib\site-packages\sklearn\utils\validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\Michael\Anaconda2\envs\gl-env\lib\site-packages\sklearn\utils\validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [25]:
del df_data_validation["Name"]
del df_data_validation["Ticket"]
del df_data_validation["Cabin"]
del df_data_validation["Embarked"]

# Go back to SFrame

In [26]:
data = gl.SFrame(df_data)
data_for_validation = gl.SFrame(df_data_validation)

This non-commercial license of GraphLab Create for academic use is assigned to docteur.zuf@gmail.com and will expire on May 23, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\Michael\AppData\Local\Temp\graphlab_server_1489669403.log.0


# Split data

In [27]:
(train, test) = data.random_split(.8)

# Build k-NN model

D'après mon étude précédente je sais que les paramètres les plus importants sont le sexe, l'age, la classe et le prix du billet

In [28]:
target = "Survived"
features = ["Pclass_n", "Age_n", "SibSp_n", "Fare_n", "Sex_n"]
#features = ["Age_n", "Fare_n", "Pclass_n", "Sex_n"]

In [29]:
#model = gl.nearest_neighbors.create(train,label="PassengerId" ,features=features, method='brute_force')
model = gl.nearest_neighbors.create(train, features = features)

#model = gl.random_forest_regression.create(train, target=target, features=features)

Starting ball tree nearest neighbors model training.

+------------+--------------+

| Tree level | Elapsed Time |

+------------+--------------+

| 0          | 988us        |

+------------+--------------+

In [30]:
model.summary()

Class                          : NearestNeighborsModel

Attributes
----------
Method                         : ball_tree
Number of distance components  : 1
Number of examples             : 734
Number of feature columns      : 5
Number of unpacked features    : 5
Total training time (seconds)  : 1.0452

Ball Tree Attributes
--------------------
Tree depth                     : 1
Leaf size                      : 1000



Apply model on the test set

In [31]:
k=5

In [32]:
knn_test = model.query(test,k=k)

+--------------+-------------+--------------+

| Query points | % Complete. | Elapsed Time |

+--------------+-------------+--------------+

| 1            | 0.5         | 0us          |

| Done         |             | 2.001ms      |

+--------------+-------------+--------------+

In [33]:
knn_test

query_label,reference_label,distance,rank
0,253,0.000138192396607,1
0,20,0.0125003196592,2
0,290,0.0125003196592,3
0,291,0.0125003196592,4
0,538,0.0125013985194,5
1,672,0.00261882399051,1
1,85,0.0140064993901,2
1,253,0.0140433195876,3
1,459,0.0257141739288,4
1,339,0.0257141739288,5


In [34]:
for i in range(k):
    print test["Survived"]["PassengerId"==knn_test["reference_label"][i]]

1
1
1
1
1


In [35]:
slices = []
for i in range(0,k*3,k):
#for i in range(int(len(knn_test)/6)):
    slice = []
    for j in range(k):
                ref = knn_test["reference_label"][i+j]
                slice.append(ref)              
    slices.append(slice)
    print slices

[[253L, 20L, 290L, 291L, 538L]]
[[253L, 20L, 290L, 291L, 538L], [672L, 85L, 253L, 459L, 339L]]
[[253L, 20L, 290L, 291L, 538L], [672L, 85L, 253L, 459L, 339L], [698L, 570L, 29L, 637L, 392L]]


In [ ]:
#Chaque ligne de "slices" est l'indice des k plus proches voisins
slices

In [ ]:
slices[0][0]

In [36]:
test_slices = pd.DataFrame(slices)
test_slices

,0,1,2,3,4
0,253,20,290,291,538
1,672,85,253,459,339
2,698,570,29,637,392


In [37]:
#Maintenant je calcule la moyenne de "survived" pour chaque item de "slices"

In [39]:
df_test = test.to_dataframe()
df_test.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Age_n,Pclass_n,SibSp_n,Fare_n,Sex_n
0,3,1,3,female,26.0,0,0,7.9250,0.3250,1.0,0.000,0.015469,1.0
1,9,1,3,female,27.0,0,2,11.1333,0.3375,1.0,0.000,0.021731,1.0
2,11,1,3,female,4.0,1,1,16.7000,0.0500,1.0,0.125,0.032596,1.0
3,18,1,2,male,29.0,0,0,13.0000,0.3625,0.5,0.000,0.025374,0.0
4,23,1,3,female,15.0,0,0,8.0292,0.1875,1.0,0.000,0.015672,1.0


In [40]:
np.mean([train["Survived"][knn_test["reference_label"][i]] for i in range(k)])

1.0

In [41]:
knn_test

query_label,reference_label,distance,rank
0,253,0.000138192396607,1
0,20,0.0125003196592,2
0,290,0.0125003196592,3
0,291,0.0125003196592,4
0,538,0.0125013985194,5
1,672,0.00261882399051,1
1,85,0.0140064993901,2
1,253,0.0140433195876,3
1,459,0.0257141739288,4
1,339,0.0257141739288,5


In [42]:
predictions = []
for i in range(0,len(knn_test),k):
#for i in range(int(len(knn_test)/6)):
    slice = []
    sum=0
    for j in range(k):
                ref = knn_test["reference_label"][i+j]
                sum+=train["Survived"][ref]
                #print train["Survived"][ref]
    pred = round((sum/k))
    #print "Prediction: ", pred
    predictions.append(pred)

In [ ]:
#Je compare mes prédictions avec les données

In [43]:
len(predictions), len(test)

(157, 157)

In [44]:
test["Prediction"]=predictions

In [45]:
test

PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Age_n,Pclass_n,SibSp_n
3,1,3,female,26.0,0,0,7.925,0.325,1.0,0.0
9,1,3,female,27.0,0,2,11.1333,0.3375,1.0,0.0
11,1,3,female,4.0,1,1,16.7,0.05,1.0,0.125
18,1,2,male,29.0,0,0,13.0,0.3625,0.5,0.0
23,1,3,female,15.0,0,0,8.0292,0.1875,1.0,0.0
24,1,1,male,28.0,0,0,35.5,0.35,0.0,0.0
26,1,3,female,38.0,1,5,31.3875,0.475,1.0,0.125
28,0,1,male,19.0,3,2,263.0,0.2375,0.0,0.375
30,0,3,male,25.0,0,0,7.8958,0.3125,1.0,0.0
32,1,1,female,38.0,1,0,146.5208,0.475,0.0,0.125


In [46]:
accuracy = ((test["Prediction"]==test["Survived"]).sum())/len(test)
accuracy

0.7898089171974523

# Appliquons maintenant le modèle sur les données de validation du concours

In [47]:
knn_for_validation = model.query(data_for_validation,k=k)

+--------------+-------------+--------------+

| Query points | % Complete. | Elapsed Time |

+--------------+-------------+--------------+

| 1            | 0           | 1.001ms      |

| Done         |             | 6.004ms      |

+--------------+-------------+--------------+

In [48]:
predictions_valid = []
for i in range(0,len(knn_for_validation),k):
#for i in range(int(len(knn_test)/6)):
    slice = []
    sum=0
    for j in range(k):
                ref = knn_for_validation["reference_label"][i+j]
                sum+=train["Survived"][ref]
                #print train["Survived"][ref]
    pred = int(round((sum/k)))
    #print "Prediction: ", pred
    predictions_valid.append(pred)

In [49]:
len(predictions_valid), len(data_for_validation)

(418, 418)

In [50]:
data_for_validation["Predictions"] = predictions_valid

In [51]:
data_for_validation

PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Age_n,Pclass_n,SibSp_n,Fare_n
892,3,male,34.5,0,0,7.8292,0.453947368421,1.0,0.0,0.0152815806712
893,3,female,47.0,1,0,7.0,0.618421052632,1.0,0.125,0.0136630900601
894,2,male,62.0,0,0,9.6875,0.815789473684,0.5,0.0,0.0189087407081
895,3,male,27.0,0,0,8.6625,0.355263157895,1.0,0.0,0.0169080739493
896,3,female,22.0,1,1,12.2875,0.289473684211,1.0,0.125,0.0239836027304
897,3,male,14.0,0,0,9.225,0.184210526316,1.0,0.0,0.0180060008292
898,3,female,30.0,0,0,7.6292,0.394736842105,1.0,0.0,0.0148912066695
899,2,male,26.0,1,1,29.0,0.342105263158,0.5,0.125,0.0566042302488
900,3,female,18.0,0,0,7.2292,0.236842105263,1.0,0.0,0.014110458666
901,3,male,21.0,2,0,24.15,0.276315789474,1.0,0.25,0.0471376607072


In [52]:
result = gl.SFrame({"PassengerId":[],"Survived":[]})
result

PassengerId,Survived


In [53]:
#result = gl.SFrame({'PassengerId':data_for_validation['PassengerId'],"Survived":predictions_valid})
a = gl.SArray(data_for_validation['PassengerId'])
b = gl.SArray(predictions_valid)
result = gl.SFrame([a,b])

In [54]:
result.rename({"X1":"PassengerId", "X2":"Survived"})

PassengerId,Survived
892,0
893,0
894,0
895,0
896,1
897,0
898,0
899,0
900,1
901,0


#features = ["Age_n", "Fare_n", "Pclass_n", "Sex_n"]

In [ ]:
#result.export_csv("gender_submission10.csv") #3-NN model score=0.76

In [ ]:
#result.export_csv("gender_submission11.csv") #7-NN model (coef_femme = 10) score = 0.75

In [ ]:
#result.export_csv("gender_submission12.csv") #3-NN model (coef_femme = 10) 0.74

In [ ]:
#result.export_csv("gender_submission13.csv") #5-NN model (coef_femme = 2) 0.75

features = ["Pclass_n", "Age_n", "SibSp_n", "Fare_n", "Sex_n"]


In [ ]:
result.export_csv("gender_submission14.csv") #5-NN model (coef_femme = 2) 0.72 